# Intro to sciviz -- 3D graphics in jupyter notebooks

Note: We might do things out of order!  

1. ipyvolume
1. yt (next week!)

Import usual things:

In [1]:
import numpy as np
import pandas as pd

# 1. Using ipyvolume

Let's make a quick test plot with ipyvolume:

In [2]:
import ipyvolume

In [3]:
x, y, z = np.random.random((3, 100)) # 3 dimensions, 100 points total
ipyvolume.quickscatter(x, y, z, size=1, marker="sphere")

In [4]:
gfile = '/Users/jnaiman/Downloads/galaxyFiles/outarrsnap_001_fac1n3.txt'
# unzip galaxyFiles.zip

In [5]:
# where is solver libs?
from sys import path
path.append('/Users/jnaiman/Downloads/galaxyFiles/') # this is where I've unpacked my zip

from solverlibs import read_in_galaxy_data

t_h, r_h, v_h, e_h, N, part_type = \
     read_in_galaxy_data(gfile)

In [6]:
r_h.shape

(82, 3, 500)

In [7]:
# one timestep
nT = 0

x, y, z = r_h[:,0,nT], r_h[:,1,nT], r_h[:,2,nT]

In [8]:
ipyvolume.quickscatter(x, y, z, size=1, marker="sphere")

This can take a super long time!!!!  Probably don't run this one:

In [9]:
# all timesteps:

###x, y, z = r_h[:,0,:].ravel(), r_h[:,1,:].ravel(), r_h[:,2,:].ravel()

In [10]:
#ipyvolume.quickscatter(x, y, z, size=1, marker="sphere")

What about velocity vectors?  Let's make a full figure object to plot upon:

In [11]:
ipyvolume.figure()
for i in range(v_h.shape[0]): # loop every particle
    ipyvolume.quiver(r_h[i,0,:], 
                      r_h[i,1,:],
                      r_h[i,2,:],
                     v_h[i,0,:], 
                      v_h[i,1,:],
                      v_h[i,2,:], 
                     size=1) # also, if things look too crowded, we can also make the arrows themselves smaller
ipyvolume.show()

Not much better, hard to see, can try with some subsampling:

In [12]:
r_h.shape

(82, 3, 500)

In [13]:
step = 10 # plot ever "step"th velocity vector
# also, length of arrays in time-axis
N = v_h.shape[2]

ipyvolume.figure()
for i in range(v_h.shape[0]): # loop every particle
    ipyvolume.quiver(r_h[i,0,0:N:step], # plot subsampled x/y/z
                      r_h[i,1,0:N:step],
                      r_h[i,2,0:N:step],
                     v_h[i,0,0:N:step], # with subsampled vectors vx/vy/vz
                      v_h[i,1,0:N:step],
                      v_h[i,2,0:N:step], 
                     size=1) # also, if things look too crowded, we can also make the arrows themselves smaller
ipyvolume.show()

## 3d animations


Let's now figure out how to make an animation in 3D, and then save it for ourselves!  To do this, we'll need to format our data specifically as [time, position].  This is yet another example of where a lot of data-viz is data-formatting!

Onto formatting our data as [time, position], we'll do this with a `.T` function that "flips" our data shape.  `.T` stands for "transposing":

In [14]:
# for example, for particle 0:
print(r_h[:,0,:].shape, r_h[:,0,:].T.shape)

(82, 500) (500, 82)


Down sample our velocity data for easier plotting:

In [15]:
step = 10 # only do every 10 steps
# also, length of arrays in time
N = v_h.shape[2]

# subsample to make more managable
r = r_h[:,:,0:N:step]
v = v_h[:,:,0:N:step]

print(r_h.shape, r.shape, r[:,2,:].T.shape)

(82, 3, 500) (82, 3, 50) (50, 82)


In [16]:
ipyvolume.figure()

s = ipyvolume.scatter(r[:,0,:].T, r[:,1,:].T, r[:,2,:].T, 
                      marker='sphere')

ani = ipyvolume.animation_control(s, interval=200)

ipyvolume.show()

**Astronomy Note:** This is actually a simulation of two galaxies merging.  *However*, its done with only a few particles so you can see the galaxies "exploding" rather than merging.  Try one of the larger files in the file folder and see how it changes!

### ipyvolume + ipywidgets

Now let's combine the powers of widgets and ipyvolume to explore our datasets in 3D.

Let's only plot a few points on a scatter plot:

In [17]:
step = 10 # only do every 10th timestep
# also, length of arrays
N = v_h.shape[2] # full time

# decimate again
r = r_h[:,:,0:N:step]
v = v_h[:,:,0:N:step]

Let's format for our scatter plot again, with some sub-sampling for speed:

In [18]:
x = r[:,0,0:N:step].ravel()
y = r[:,1,0:N:step].ravel()
z = r[:,2,0:N:step].ravel()

Reconstruct our scatter plot with every 100th point:

In [19]:
ipyvolume.figure()

s = ipyvolume.scatter(x, y, z, 
                      marker='sphere')

ipyvolume.show()

Now let's create this plot but we won't show it:

In [20]:
ipyvolume.figure()

s = ipyvolume.scatter(x, y, z, 
                      marker='sphere')

We can link widgets to ths to control the size & color of our points.  Let's use a `FloatSlider` widget and a `ColorPicker` widget:

In [21]:
import ipywidgets

In [22]:
size = ipywidgets.FloatSlider(min=0, max=30, step=0.1)
color = ipywidgets.ColorPicker()

In [23]:
size

FloatSlider(value=0.0, max=30.0)

In [24]:
color

ColorPicker(value='black')

In [25]:
size.value

0.0

In [26]:
color.value

'black'

Now we'll use a widget function we haven't used before called `jslink`.  With this we can link attributes of our scatter plot like the scatter *size* and *color* with the *value*s of each of our widgets:

In [27]:
s.size

array(2)

In [28]:
s.color

array('red', dtype='<U3')

In [29]:
ipywidgets.jslink((s, 'size'), (size, 'value'))
ipywidgets.jslink((s, 'color'), (color, 'value'))

Link(source=(Scatter(color_selected=array('white', dtype='<U5'), geo='sphere', line_material=ShaderMaterial(),…

Finally, well put all these things in a column - our plot, then our two linked widgets:

In [30]:
ipywidgets.VBox([ipyvolume.gcc(), size,  color])

### export your 3D object to a webpage

Finally, we might want to embed our creations on the web somewhere.  The first step is to make an ```html``` file from our in-python widgets.  Luckily, there is a function for that!  Instead of displaying our `VBox`, let's save it as an object:

In [31]:
myVBox = ipywidgets.VBox([ipyvolume.gcc(), size,  color])

Now we'll need to do a few layout things so that the box won't be super small on our embedded webpage (try it without and see what happens!)

In [32]:
ipyvolume.embed.layout = myVBox.children[1].layout
ipyvolume.embed.layout.min_width = "400px"

Finally we'll "embed" our interactive plot within a webpage:

In [33]:
# NOTE!!!! offline=False OR True may or may not work... depends
#ipyvolume.embed.embed_html("myPage.html", myVBox, offline=False, devmode=False)
ipyvolume.embed.embed_html("myPage.html", myVBox, offline=True, devmode=False)

This webpage is locally stored, but you can open it up in your default browser with (on a Mac):

In [34]:
!open myPage.html

### Bonus: animation + widgets

This is just the code for how this looks feel free to poke around at it if you are interested!

First, a test plot:

In [35]:
step = 10 # only do every 10th timestep
# also, length of arrays
N = r_h.shape[2] # full time
# decimate again
r = r_h[:,:,0:N:step]


ipyvolume.figure()

size = ipywidgets.FloatSlider(min=0, max=30, step=0.1, description='Size of Particles')

s = ipyvolume.scatter(r[:,0,:].T, r[:,1,:].T, r[:,2,:].T, 
                      marker='sphere')

ipywidgets.jslink((s, 'size'), (size, 'value'))
ani = ipyvolume.animation_control(s, interval=200)
ipywidgets.VBox([ipyvolume.gcc(), size])

Save with animation:

In [37]:
ipyvolume.figure()

size = ipywidgets.FloatSlider(min=0, max=30, step=0.1, description='Size of Particles')

s = ipyvolume.scatter(r[:,0,:].T, r[:,1,:].T, r[:,2,:].T, 
                      marker='sphere')

ipywidgets.jslink((s, 'size'), (size, 'value'))
ani = ipyvolume.animation_control(s, interval=200)

# export to web
myVboxOut = ipywidgets.VBox([ipyvolume.gcc(), size])
# if we don't do this, the plot will be really tiny in the standalone html
ipyvolume.embed.layout = myVboxOut.children[1].layout
ipyvolume.embed.layout.min_width = "400px"
#ipyvolume.embed.embed_html("myPage_withAni.html", myVboxOut, offline=False, devmode=False)
ipyvolume.embed.embed_html("myPage_withAni.html", myVboxOut, offline=True, devmode=False)

Check it out:

In [38]:
!open myPage_withAni.html

**NOTE!!  Online seems to not work now...**

Try for export to online as well:

In [41]:
ipyvolume.figure()

size = ipywidgets.FloatSlider(min=0, max=30, step=0.1, description='Size of Particles')

s = ipyvolume.scatter(r[:,0,:].T, r[:,1,:].T, r[:,2,:].T, 
                      marker='sphere')

ipywidgets.jslink((s, 'size'), (size, 'value'))
ani = ipyvolume.animation_control(s, interval=200)

# export to web
myVboxOut = ipywidgets.VBox([ipyvolume.gcc(), size])
# if we don't do this, the plot will be really tiny in the standalone html
ipyvolume.embed.layout = myVboxOut.children[1].layout
ipyvolume.embed.layout.min_width = "400px"
ipyvolume.embed.embed_html("myPage_withAni_online.html", myVboxOut, offline=False, devmode=True)

In [42]:
!open myPage_withAni_online.html

Now you have a lovely image that you can upload and put on things like sweaters or whatnot.

In [38]:
# try with other files
gfile = '/Users/jnaiman/Downloads/galaxyFiles/outarrsnap_001_fac1n2.txt'
t_h, r_h, v_h, e_h, N, part_type = \
     read_in_galaxy_data(gfile)

# downgrade time sampling
step = 100